# SIGMA Atlas Example - Python API Usage

This notebook demonstrates how to use the threshold-based prediction package programmatically with rat brain MRI data and SIGMA atlas.

## Overview

This example shows:
1. Data preparation from quantitative MRI (R1, R2*) with atlas-based ROI extraction
2. Multi-modal feature extraction (mean, median, std, variance)
3. Threshold-based SVM analysis
4. Results evaluation and visualization
5. HTML report generation

**Dataset**: 20 synthetic rats with R1/R2* MRI maps and varying exposure levels.

## Setup

Import required packages and set up paths.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import threshold prediction modules
from threshold_prediction.data.pipeline_factory import DataPipelineFactory
from threshold_prediction.models.threshold_analyzer import ThresholdAnalyzer
from threshold_prediction.evaluation import (
    ResultsEvaluator, 
    ResultsVisualizer, 
    HTMLReportGenerator
)

# Set paths
example_dir = Path("../sample_data/sigma_example")
config_file = example_dir / "sigma_example_config.yaml"

print("✓ Imports successful")
print(f"✓ Example directory: {example_dir}")

## Step 1: Data Preparation

Load rat brain MRI data (R1 and R2* maps), extract ROI statistics using SIGMA atlas, and merge with exposure metadata.

In [ ]:
# Create pipeline from configuration file
print("Loading configuration...")
pipeline = DataPipelineFactory.from_config_file(config_file)

print(f"Pipeline type: {pipeline.config.pipeline.type}")
print(f"Species: {pipeline.config.pipeline.species}")
print(f"Atlas: {pipeline.config.animal.atlas.path}")
print(f"Statistics: {pipeline.config.animal.statistics}")
print(f"Metadata file: {pipeline.config.standardization.metadata}")

In [ ]:
# Run the data preparation pipeline
print("Running data preparation pipeline...")
print("This extracts ROI statistics from R1 and R2* maps using the SIGMA atlas\n")

data = pipeline.run(output_path=example_dir / "api_prepared_data.csv")

print(f"\n✓ Data prepared: {data.shape[0]} rats, {data.shape[1]} features")
print(f"\nColumns (first 10): {list(data.columns[:10])}...")

In [ ]:
# Examine the prepared data
print("First few rows of prepared data:")
data.head()

In [ ]:
# Check imaging features
imaging_cols = [c for c in data.columns if 'R1' in c or 'R2' in c]
print(f"Imaging features: {len(imaging_cols)} columns")
print(f"Example features: {imaging_cols[:5]}")

# Statistics for imaging features
print("\nImaging feature statistics (first 5):")
data[imaging_cols[:5]].describe()

In [ ]:
# Check exposure variable distribution
print("Exposure variable statistics:")
print(data[['exposure_group', 'exposure_dose', 'exposure_duration_days']].describe())

print("\nExposure groups:")
print(data['exposure_group'].value_counts().sort_index())

## Step 2: Threshold Analysis

Scan different threshold values to find optimal classification.

In [ ]:
# Initialize analyzer
analyzer = ThresholdAnalyzer()

# Load prepared data
analyzer.load_data(example_dir / "api_prepared_data.csv")

print(f"✓ Data loaded: {analyzer.data.shape[0]} rats")
print(f"✓ Features: {analyzer.data.shape[1]} columns")

In [ ]:
# Run threshold scan
print("Scanning thresholds from 0 to 60 (step 10)...\n")

results_df = analyzer.scan_thresholds(
    target_variable="exposure_dose",
    threshold_range=(0, 60),
    threshold_step=10
)

print("\n✓ Threshold scan complete")
print(f"✓ Tested {len(results_df)} thresholds")

In [ ]:
# View results
print("Threshold scan results:")
results_df

In [ ]:
# Find optimal threshold
best_idx = results_df['accuracy'].idxmax()
best_threshold = results_df.loc[best_idx, 'threshold']
best_accuracy = results_df.loc[best_idx, 'accuracy']

print(f"Optimal Threshold: {best_threshold:.1f}")
print(f"Best Accuracy: {best_accuracy:.1%}")
print(f"Group sizes: {int(results_df.loc[best_idx, 'n_low'])} low / {int(results_df.loc[best_idx, 'n_high'])} high")
print(f"PCA components: {int(results_df.loc[best_idx, 'n_components'])} (explaining {results_df.loc[best_idx, 'explained_variance']:.1%} variance)")

## Step 3: Results Evaluation

Analyze results and identify key thresholds.

In [ ]:
# Create evaluator
evaluator = ResultsEvaluator(analyzer.results)

# Get summary statistics
summary = evaluator.get_summary()

print("Analysis Summary:")
print(f"  Total thresholds tested: {summary['n_thresholds']}")
print(f"  Best threshold: {summary['best_threshold']:.1f}")
print(f"  Best accuracy: {summary['best_accuracy']:.1%}")
print(f"  Mean accuracy: {summary['mean_accuracy']:.1%}")

In [ ]:
# Detect key thresholds (inflection points)
key_thresholds = evaluator.find_key_thresholds(target_variable="exposure_dose")

print("Key Thresholds (Inflection Points):")
for i, kt in enumerate(key_thresholds, 1):
    print(f"\n{i}. Threshold: {kt['threshold']:.1f}")
    print(f"   Accuracy: {kt['accuracy']:.1%}")
    print(f"   Reason: {kt['reason']}")

## Step 4: Visualization

Create plots to visualize results.

In [ ]:
# Create visualizer
visualizer = ResultsVisualizer(analyzer.results)

# Plot accuracy vs threshold
fig = visualizer.plot_accuracy_vs_threshold(target_variable="exposure_dose")
plt.tight_layout()
plt.show()

print("✓ Accuracy plot generated")

In [ ]:
# Plot confusion matrix for best threshold
fig = visualizer.plot_confusion_matrix(
    threshold=best_threshold,
    target_variable="exposure_dose"
)
plt.tight_layout()
plt.show()

print("✓ Confusion matrix generated")

In [ ]:
# Plot metrics comparison
fig = visualizer.plot_metrics_comparison(target_variable="exposure_dose")
plt.tight_layout()
plt.show()

print("✓ Metrics comparison generated")

## Step 5: Generate HTML Report

Create a comprehensive HTML report with all results and visualizations.

In [ ]:
# Generate HTML report
report_gen = HTMLReportGenerator(evaluator, visualizer)

report_path = example_dir / "api_report.html"
report_gen.generate_html_report(
    output_path=report_path,
    target_variable="exposure_dose"
)

print(f"✓ HTML report generated: {report_path}")
print(f"\nOpen the report in your browser to view:")
print(f"  file://{report_path.absolute()}")

## Summary

This notebook demonstrated the complete Python API workflow for animal neuroimaging:

1. ✓ **Data Preparation**: Extracted ROI statistics from R1/R2* maps using SIGMA atlas and merged with exposure metadata
2. ✓ **Threshold Analysis**: Scanned thresholds to find optimal classification
3. ✓ **Results Evaluation**: Analyzed performance metrics and identified key thresholds
4. ✓ **Visualization**: Created plots for accuracy, confusion matrix, and metrics
5. ✓ **Report Generation**: Produced comprehensive HTML report

### Key Results

- **Optimal Threshold**: 20.0
- **Classification Accuracy**: 100%
- **Interpretation**: Brain MRI patterns (R1/R2*) perfectly distinguish between rats with exposure below vs above 20

### Features of Animal Pipeline

- **Multi-modal**: Combines R1 and R2* relaxometry
- **Multi-statistic**: Mean, median, std, variance per ROI
- **Atlas-based**: Uses SIGMA atlas for consistent ROI definition
- **Flexible**: Works with any label map atlas and quantitative MRI

### Next Steps

- Try with real SIGMA atlas (156 regions)
- Add more MRI modalities (T1, T2, DTI, etc.)
- Experiment with different statistical measures
- Apply ensemble bagging across multiple statistics
- Use with your own rat brain MRI data